In [10]:
import requests
from bs4 import BeautifulSoup as bs
import yaml
from IPython.display import display
from time import sleep
import re
from tqdm import tqdm
import pandas as pd

from typing import List, Dict, Any, Set, Tuple, Union, Optional


In [1]:
import os
os.chdir("..")
print(os.getcwd())

from classes import *
len(speakings)


c:\Users\이찬행\OneDrive - 충남대학교\문서\GitHub\snu-factcheck-analytics


3292

In [32]:
fc_dicts = {}
for key, value in tqdm(speaks_dict.items()):
    sp_info = value.copy()
    factchecks = sp_info.pop('factchecks')
    for id, fc in factchecks.items():
        fc_info = fc.copy()
        fc_info.update(sp_info)
        fc_dicts[id] = fc_info
print(len(fc_dicts))


100%|██████████| 3292/3292 [00:00<00:00, 411003.09it/s]

3549


In [33]:
fc_dicts[1]

{'content': "검증내용 <앵커>대선 국면에 접어들면서 저희 사실은 코너에선 선거 관련 가짜 뉴스와 대선 후보 관련 논란을 집중 검증하고 있습니다. 마침 저희 사실은 홈페이지에 지지율 1위 문재인 민주당 후보의 아들 취업 특혜 논란을 따져봐 달라는 요청이 들어와서, 오늘(25일)은 그 문제를 한번 집중적으로 따져 보겠습니다.박세용 기자, 먼저 논란의 핵심부터 정리해주시죠.<기자>문재인 후보 아들이 노무현 정부 시절인 2007년 1월에 한국고용정보원이라는 공기업에 합격합니다.당시 고용정보원의 원장이 청와대에서 문재인 후보와 같이 근무했던 사람이거든요.그래서 아버지 덕에 취업한 거 아니냐는 의혹이\xa0나오는 겁니다.<앵커>이런 의혹이 자꾸 나오니까 선관위는 최근 이런 의혹들은 모두 허위라고 하면서 관련 게시물을 단속하고 있잖아요?<기자>선관위가 지금 단속하는 건 악의적인 가짜 뉴스입니다.문 후보 아들 1명만 지원해서 혼자 합격했다, 또 5급 공무원 자리다, 이런 글들이\xa0돌고 있는데\xa0사실이 아니라 거짓이라는 겁니다.실제로는 2명 지원해서 2명 합격했고, 공무원이 아니라 공기업 일반직입니다.<앵커>이런 의혹에 대해서 문재인 후보 측은 2007년과 2010년 이미 두 번 다 감사를 받았고, 거기서 문제없는 걸로 나왔다고 해명하고 있는 거죠?<기자>취재를 해보니까 그 부분은 절반만 사실이었습니다.2007년에 고용정보원이 노동부에서 감사를 받은 건 사실입니다.당시 감사 결과를 보시면 문 후보 아들이\xa0경쟁 없이 채용된 건 사실이지만, 특혜를 받았다는\xa0확증은 발견되지 않았다는 겁니다.<앵커>2007년은 이랬고, 2010년 감사 내용은 어떤가요?<기자>2010년 이명박 정부 때도 감사를 받았다, 만약에 특혜가 사실이었다면 이명박 정부가 이 사안을 가만히 뒀겠느냐는 얘긴데, 사실 당시 2010년 감사는 성격이\xa0완전히 다릅니다.당시 감사는 야당이었던 민주당 의원들이 요구한 겁니다.이명박 정부가 임명한 고용정보원장이 채용 특혜에 연루됐다며 감사를 

In [18]:
for i, j in fc_dicts.items():
    if j != None:
        print(i, j)
        break